In [7]:
import yfinance as yf
import pandas as pd

def fetch_ohlcv(ticker, start_date, end_date):
    # Fetch OHLCV data from Yahoo Finance using yfinance library
    data = yf.download(ticker, start=start_date, end=end_date)
    # Create a pandas DataFrame
    df = pd.DataFrame(data)
    return df

In [24]:
def calculate_bollinger_bands(df,period=14,std_deviaton=2):
    _dataset = df.copy()
    _dataset['MB'] = _dataset["Adj Close"].rolling(period).mean()
    _dataset['UB'] =_dataset['MB'] + 2*_dataset["Adj Close"].rolling(period).std(ddof=0)
    _dataset['LB'] = _dataset['MB'] - 2*_dataset["Adj Close"].rolling(period).std(ddof=0)
    _dataset["BB_WIDTH"] = _dataset["UB"]-_dataset["LB"]
    return _dataset

In [19]:
import pandas as pd

def calculate_sma(df, period=20, column='Adj Close'):
    _dataset = df.copy()
    # Calculate simple moving average
    moving_average = _dataset[column].rolling(period).mean()
    # Add moving average to DataFrame
    _dataset['SMA'] = moving_average
    return _dataset

In [73]:
# Example usage
ticker = "ITC.NS"
start_date = "2023-01-01"
end_date = "2023-6-30"

ohlcv_data = fetch_ohlcv(ticker, start_date, end_date)
ohlcv_data = calculate_bollinger_bands(ohlcv_data)
ohlcv_data = calculate_sma(ohlcv_data,50)
test_data = ohlcv_data[["Adj Close","SMA","LB","MB","UB"]]

[*********************100%***********************]  1 of 1 completed


In [76]:
print(test_data.to_markdown())

| Date                |   Adj Close |     SMA |      LB |      MB |      UB |
|:--------------------|------------:|--------:|--------:|--------:|--------:|
| 2023-01-02 00:00:00 |     325.676 | nan     | nan     | nan     | nan     |
| 2023-01-03 00:00:00 |     323.963 | nan     | nan     | nan     | nan     |
| 2023-01-04 00:00:00 |     319.952 | nan     | nan     | nan     | nan     |
| 2023-01-05 00:00:00 |     326.263 | nan     | nan     | nan     | nan     |
| 2023-01-06 00:00:00 |     327.828 | nan     | nan     | nan     | nan     |
| 2023-01-09 00:00:00 |     330.764 | nan     | nan     | nan     | nan     |
| 2023-01-10 00:00:00 |     326.214 | nan     | nan     | nan     | nan     |
| 2023-01-11 00:00:00 |     324.648 | nan     | nan     | nan     | nan     |
| 2023-01-12 00:00:00 |     322.74  | nan     | nan     | nan     | nan     |
| 2023-01-13 00:00:00 |     321.664 | nan     | nan     | nan     | nan     |
| 2023-01-16 00:00:00 |     323.083 | nan     | nan     | nan   

In [75]:
position = False
buy_price = 0
results = []

# Stop-loss parameters
stop_loss_enabled = True
stop_loss_percent = 0.03  # 3% stop loss

for index, row in test_data.iterrows():
    if position:
        if row["Adj Close"] >= row["UB"] or (stop_loss_enabled and row['Adj Close'] == row["LB"]):
            # Sell signal
            position = False
            sell_price = row['Adj Close']
            profit = sell_price - buy_price
            results.append(profit)
    else:
        if row["Adj Close"] > row['SMA'] and row["Adj Close"] > row["MB"]:
            # Buy signal
            position = True
            buy_price = row["Adj Close"]
cumulative_profits = [0]
for profit in results:
    cumulative_profits.append(cumulative_profits[-1] + profit)

# Print results
print(results)
print("Cumulative Profits:", cumulative_profits)

[3.627655029296875, 8.597015380859375, 31.804046630859375, 16.59771728515625, 5.466339111328125, 3.20001220703125]
Cumulative Profits: [0, 3.627655029296875, 12.22467041015625, 44.028717041015625, 60.626434326171875, 66.0927734375, 69.29278564453125]
